[View in Colaboratory](https://colab.research.google.com/github/AhmadHAW/haw/blob/master/rnn_mul2_example.ipynb)

In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.8.0


### Step 1: Generate sample equations

In [4]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [5]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [6]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789* '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}*{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a * b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS *2 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [11]:
questions[0]

'15*7   '

In [12]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS *2, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS *2)


Vectorization...


In [13]:
len(x[0])

7

In [14]:
len(questions[0])

7

In [15]:
questions[0]

'15*7   '

In [16]:
x[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [17]:
y[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [18]:
expected[0]

'105   '

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [19]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 6, 12)
Validation Data:
(5000, 7, 12)
(5000, 6, 12)


### Step 3: Create Model

In [20]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [25]:
from keras.models import Sequential
from keras import layers

# Try replacing LSTM, GRU, or SimpleRNN.
# RNN = layers.LSTM
RNN = layers.SimpleRNN
# RNN = layers.GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS *2))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 128)               18048     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 6, 128)            32896     
_________________________________________________________________
classifier (Dense)           (None, 6, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.predict(np.array([x_val[0]]))

array([[[0.14441244, 0.04574936, 0.05137076, 0.05948979, 0.10691672,
         0.06883045, 0.05966745, 0.06091909, 0.08527803, 0.08932285,
         0.09715392, 0.13088916],
        [0.25856924, 0.04145455, 0.04642598, 0.06040891, 0.11475559,
         0.04878478, 0.04404378, 0.04076315, 0.06858587, 0.05803497,
         0.13399154, 0.08418161],
        [0.17238353, 0.03802868, 0.07444333, 0.04973732, 0.13406482,
         0.04826393, 0.06626695, 0.05739867, 0.05773523, 0.04530224,
         0.18135525, 0.07502005],
        [0.15975589, 0.02953937, 0.08972628, 0.05411725, 0.11526451,
         0.05424192, 0.07472758, 0.05803854, 0.05149968, 0.03990347,
         0.17499048, 0.09819505],
        [0.18563895, 0.03344136, 0.07740403, 0.04168507, 0.08845405,
         0.04901558, 0.07457905, 0.05432715, 0.05213454, 0.05073697,
         0.20468472, 0.08789857],
        [0.18587734, 0.02947511, 0.07531032, 0.04471239, 0.08155988,
         0.04556641, 0.06754129, 0.0637871 , 0.04201061, 0.04585004,
  

In [23]:
model.predict_classes(np.array([x_val[0]]))

array([[ 6,  6,  6, 11]])

### Step 4: Train

In [27]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 267us/step - loss: 1.6972 - acc: 0.3723 - val_loss: 1.7376 - val_acc: 0.3464
Q 74*597  T 44178  ☒ 47672 
Q 27*965  T 26055  ☒ 12875 
Q 149*5   T 745    ☒ 145   
Q 491*198 T 97218  ☒ 16008 
Q 97*522  T 50634  ☒ 14668 
Q 959*97  T 93023  ☒ 48875 
Q 130*863 T 112190 ☒ 137800
Q 18*169  T 3042   ☒ 1248  
Q 39*377  T 14703  ☒ 12831 
Q 666*12  T 7992   ☒ 1036  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
18816/45000 [===========>..................] - ETA: 6s - loss: 1.5634 - acc: 0.4096

45000/45000 [==============================] - 12s 265us/step - loss: 1.5435 - acc: 0.4143 - val_loss: 1.6677 - val_acc: 0.3621
Q 436*426 T 185736 ☒ 131062
Q 35*690  T 24150  ☒ 28200 
Q 800*96  T 76800  ☒ 62200 
Q 633*79  T 50007  ☒ 36871 
Q 214*964 T 206296 ☒ 111562
Q 809*54  T 43686  ☒ 48880 
Q 293*71  T 20803  ☒ 11811 
Q 733*21  T 15393  ☒ 10811 
Q 761*288 T 219168 ☒ 181556
Q 20*256  T 5120   ☒ 1200  

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
30592/45000 [===================>..........] - ETA: 3s - loss: 1.4813 - acc: 0.4319

45000/45000 [==============================] - 12s 267us/step - loss: 1.4696 - acc: 0.4344 - val_loss: 1.5969 - val_acc: 0.3782
Q 839*936 T 785304 ☒ 732116
Q 616*134 T 82544  ☒ 10406 
Q 150*32  T 4800   ☒ 1000  
Q 312*37  T 11544  ☒ 12146 
Q 620*29  T 17980  ☒ 17440 
Q 78*693  T 54054  ☒ 57566 
Q 14*378  T 5292   ☒ 1128  
Q 133*909 T 120897 ☒ 127177
Q 67*665  T 44555  ☒ 41455 
Q 410*363 T 148830 ☒ 119760

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
33664/45000 [=====================>........] - ETA: 2s - loss: 1.4197 - acc: 0.4488

45000/45000 [==============================] - 12s 266us/step - loss: 1.4122 - acc: 0.4514 - val_loss: 1.5467 - val_acc: 0.3951
Q 167*605 T 101035 ☒ 11575 
Q 92*553  T 50876  ☒ 48448 
Q 308*68  T 20944  ☒ 25534 
Q 559*701 T 391859 ☒ 355447
Q 74*189  T 13986  ☒ 11538 
Q 60*534  T 32040  ☒ 35840 
Q 16*29   T 464    ☒ 648   
Q 992*22  T 21824  ☒ 25878 
Q 787*97  T 76339  ☒ 65771 
Q 46*492  T 22632  ☒ 25554 

--------------------------------------------------
Iteration 5
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
34176/45000 [=====================>........] - ETA: 2s - loss: 1.3664 - acc: 0.4667

45000/45000 [==============================] - 12s 264us/step - loss: 1.3631 - acc: 0.4676 - val_loss: 1.5159 - val_acc: 0.4051
Q 166*330 T 54780  ☒ 50160 
Q 409*804 T 328836 ☒ 363344
Q 21*276  T 5796   ☒ 1474  
Q 944*201 T 189744 ☒ 297178
Q 863*287 T 247681 ☒ 239111
Q 691*26  T 17966  ☒ 15576 
Q 612*348 T 212976 ☒ 255574
Q 190*32  T 6080   ☒ 6060  
Q 40*490  T 19600  ☒ 26400 
Q 236*677 T 159772 ☒ 151576

--------------------------------------------------
Iteration 6
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
33920/45000 [=====================>........] - ETA: 2s - loss: 1.3293 - acc: 0.4790

...

45000/45000 [==============================] - 12s 264us/step - loss: 1.1649 - acc: 0.5427 - val_loss: 1.3536 - val_acc: 0.4636
Q 839*789 T 661971 ☒ 652581
Q 114*543 T 61902  ☒ 58876 
Q 30*674  T 20220  ☒ 21780 
Q 35*533  T 18655  ☒ 18865 
Q 570*346 T 197220 ☒ 185680
Q 87*918  T 79866  ☒ 80126 
Q 81*833  T 67473  ☒ 68411 
Q 726*52  T 37752  ☒ 36618 
Q 361*46  T 16606  ☒ 17182 
Q 60*601  T 36060  ☒ 36380 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
33152/45000 [=====================>........] - ETA: 2s - loss: 1.1560 - acc: 0.5476

45000/45000 [==============================] - 12s 264us/step - loss: 1.1566 - acc: 0.5473 - val_loss: 1.3455 - val_acc: 0.4716
Q 457*6   T 2742   ☒ 2522  
Q 488*150 T 73200  ☒ 66800 
Q 3*383   T 1149   ☒ 1177  
Q 545*555 T 302475 ☒ 371475
Q 970*465 T 451050 ☒ 452250
Q 39*267  T 10413  ☒ 1083  
Q 184*74  T 13616  ☒ 13052 
Q 629*41  T 25789  ☒ 25927 
Q 370*70  T 25900  ☒ 24900 
Q 76*995  T 75620  ☒ 73480 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
33920/45000 [=====================>........] - ETA: 2s - loss: 1.1431 - acc: 0.5524

45000/45000 [==============================] - 12s 263us/step - loss: 1.1425 - acc: 0.5526 - val_loss: 1.3274 - val_acc: 0.4771
Q 52*197  T 10244  ☒ 9024  
Q 295*418 T 123310 ☒ 121810
Q 29*554  T 16066  ☒ 15024 
Q 805*8   T 6440   ☒ 6840  
Q 78*904  T 70512  ☒ 77474 
Q 71*504  T 35784  ☒ 32104 
Q 20*956  T 19120  ☒ 18540 
Q 240*972 T 233280 ☒ 222180
Q 295*79  T 23305  ☒ 23405 
Q 843*22  T 18546  ☒ 19748 

--------------------------------------------------
Iteration 18
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
33152/45000 [=====================>........] - ETA: 2s - loss: 1.1345 - acc: 0.5558

45000/45000 [==============================] - 12s 264us/step - loss: 1.1340 - acc: 0.5557 - val_loss: 1.3401 - val_acc: 0.4679
Q 658*178 T 117124 ☒ 114106
Q 74*590  T 43660  ☒ 47340 
Q 481*627 T 301587 ☒ 282229
Q 393*246 T 96678  ☒ 90254 
Q 589*618 T 364002 ☒ 358106
Q 69*790  T 54510  ☒ 51410 
Q 77*702  T 54054  ☒ 50758 
Q 907*76  T 68932  ☒ 65782 
Q 190*32  T 6080   ☒ 6980  
Q 762*493 T 375666 ☒ 352276

--------------------------------------------------
Iteration 19
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
34432/45000 [=====================>........] - ETA: 2s - loss: 1.1218 - acc: 0.5618

45000/45000 [==============================] - 12s 264us/step - loss: 1.1197 - acc: 0.5626 - val_loss: 1.3097 - val_acc: 0.4859
Q 870*93  T 80910  ☒ 88330 
Q 761*71  T 54031  ☒ 59581 
Q 566*944 T 534304 ☒ 510052
Q 536*255 T 136680 ☒ 134050
Q 783*38  T 29754  ☒ 31598 
Q 690*29  T 20010  ☒ 18710 
Q 51*569  T 29019  ☒ 20507 
Q 736*65  T 47840  ☒ 41160 
Q 91*466  T 42406  ☒ 40404 
Q 551*73  T 40223  ☒ 47553 


In [30]:
x_val[0]

array([[False, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False]])

In [31]:
model.predict(np.array([x_val[0]]))

array([[[6.7103009e-08, 5.0901244e-10, 6.7835720e-07, 5.8451429e-02,
         9.3994659e-01, 1.5696103e-03, 1.0932302e-05, 2.1810696e-07,
         6.1855371e-07, 1.3284932e-06, 1.8718791e-06, 1.6600618e-05],
        [1.6752510e-08, 1.5015644e-07, 2.0230924e-01, 4.8340365e-01,
         1.4549074e-01, 3.4465164e-02, 3.7422872e-03, 1.5682701e-03,
         2.8963336e-03, 5.2932352e-03, 2.7150244e-02, 9.3680702e-02],
        [8.6850672e-11, 8.6863349e-07, 2.2704613e-01, 1.6979282e-01,
         9.6485205e-02, 8.3994567e-02, 5.8525663e-02, 4.7968410e-02,
         5.6849863e-02, 5.6156792e-02, 9.1090731e-02, 1.1208892e-01],
        [1.1314254e-10, 1.3827886e-07, 9.4474450e-02, 1.2675898e-01,
         8.3980039e-02, 6.1526664e-02, 9.0991095e-02, 7.0731133e-02,
         8.5424930e-02, 8.6577818e-02, 1.6866240e-01, 1.3087241e-01],
        [1.0930734e-08, 7.0225639e-08, 4.7242410e-02, 6.8379477e-02,
         8.3814323e-02, 1.5511125e-01, 1.3301560e-01, 6.6718102e-02,
         1.5219708e-01, 1.1652

In [0]:
model.predict_classes(np.array([x_val[0]]))

array([[ 8, 11,  5,  0]], dtype=int64)